In [10]:
!pip install numpy scipy matplotlib pydub soundfile
!pip install pydub --quiet
!apt-get install ffmpeg -y --quiet
import numpy as np, scipy.signal as sg, scipy.io.wavfile as wav
from pydub import AudioSegment
from google.colab import files
import time, math, io

Reading package lists...
Building dependency tree...
Reading state information...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [11]:
"""Implement the Overlapping-Add and Overlapping-Save algorithms in Python.
Create Python functions (50%) overlap_add(x, h, L),  overlap_save(x, h, L)
via circular convolution (using the FFT/IFFT for speedup)"""
def nextpow2(n):
    # 取得 >= n 的最小 2 的次方數（加速 FFT）
    return 1 << (n-1).bit_length()

def overlap_add(x, h, L):
    M = len(h)
    N = nextpow2(L + M - 1)
    H = np.fft.rfft(h, N)
    y = np.zeros(len(x) + M - 1, dtype=np.float32)

    for k in range(0, len(x), L):
        x_blk = x[k:k+L]
        x_blk_pad = np.pad(x_blk, (0, N - len(x_blk)))
        X = np.fft.rfft(x_blk_pad, N)
        Y_blk = np.fft.irfft(X * H, N)
        valid_len = min(N, len(y) - k)
        y[k:k+valid_len] += Y_blk[:valid_len]
    return y

def overlap_save(x, h, L):
    M = len(h)
    N = nextpow2(L + M - 1)
    H = np.fft.rfft(h, N)
    x_padded = np.concatenate([np.zeros(M-1, dtype=x.dtype), x])
    y_list = []
    for k in range(0, len(x), L):
        x_blk = x_padded[k:k+N]
        if len(x_blk) < N:
            x_blk = np.pad(x_blk, (0, N - len(x_blk)))
        X = np.fft.rfft(x_blk, N)
        Y_blk = np.fft.irfft(X * H, N)
        y_list.append(Y_blk[M-1:M-1+L]) #只保留L點
    return np.concatenate(y_list)[:len(x)+M-1]

In [12]:
"""Apply these methods to a real-world audio signal."""
from google.colab import drive
drive.mount('/content/drive')
wav_path = '/content/drive/MyDrive/DSP2025 Homework 7/seashell.wav'
sr, x = wav.read(wav_path)
print('Sample rate:', sr)
print('Shape:', x.shape, 'Dtype:', x.dtype)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Sample rate: 24000
Shape: (198144, 2) Dtype: int16


In [13]:
# 高通濾波器
h_hp = np.array([1, -1], dtype=np.float32)
# 低通濾波器
h_lp = np.array([1, 1, 1, 1, 1], dtype=np.float32)
#只要 x 是雙聲道（shape[1]==2），記得選單聲道
if x.ndim == 2:
    x = x[:, 0]
#轉 float32 並正規化 [-1, 1]
x = x.astype(np.float32) / np.iinfo(x.dtype).max

In [14]:
"""Verify the correctness of your implementations against a standard convolution
function."""
#setting Block Size L
L = 2048
#A simple high-pass filter:
y_ola_hp = overlap_add (x, h_hp, L)
y_ols_hp = overlap_save(x, h_hp, L)
y_ref_hp = np.convolve(x, h_hp, mode='full')

#A simple low-pass filter:
y_ola_lp = overlap_add (x, h_lp, L)
y_ols_lp = overlap_save(x, h_lp, L)
y_ref_lp = np.convolve(x, h_lp, mode='full')

In [15]:
def verify(tag, y_ola, y_ols, y_ref):
    print(f"\n=== {tag} ===")
    # (a) lengths
    print("lengths:", len(y_ola), len(y_ols), len(y_ref))
    # (b) allclose
    print("OLA allclose:", np.allclose(y_ola, y_ref, atol=1e-5))
    print("OLS allclose:", np.allclose(y_ols, y_ref, atol=1e-5))
    # (c) maximum absolute difference
    print("OLA max |Δ| :", np.max(np.abs(y_ola - y_ref)))
    print("OLS max |Δ| :", np.max(np.abs(y_ols - y_ref)))

verify("High-pass", y_ola_hp, y_ols_hp, y_ref_hp)
verify("Low-pass ", y_ola_lp, y_ols_lp, y_ref_lp)


=== High-pass ===
lengths: 198145 198145 198145
OLA allclose: True
OLS allclose: True
OLA max |Δ| : 8.940697e-08
OLS max |Δ| : 5.9604645e-08

=== Low-pass  ===
lengths: 198148 198148 198148
OLA allclose: True
OLS allclose: True
OLA max |Δ| : 7.4505806e-08
OLS max |Δ| : 7.0780516e-08


In [17]:
"""Compare the execution time of your overlap_add, overlap_save,
and np.convolve(x, h, mode='full') with various Signal
Length, Filter Length, and Block Size. (You can define a random signal for this
task.)"""
#隨機訊號
x = np.random.randn(2**18).astype(np.float32)
h = np.random.randn(513).astype(np.float32)
L = 2048

In [19]:
#計時器
def bench(func, *args):
    t0 = time.perf_counter()
    func(*args)
    t1 = time.perf_counter()
    return t1 - t0
print("overlap_add:", bench(overlap_add, x, h, L), "s")
print("overlap_save:", bench(overlap_save, x, h, L), "s")
print("np.convolve:", bench(np.convolve, x, h, 'full'), "s")

overlap_add: 0.024963641999875108 s
overlap_save: 0.01623768299987205 s
np.convolve: 0.016189416999623063 s


In [20]:
#various Signal Length, Filter Length, and Block Size
results = []
for L in [256, 512, 1024, 2048, 4096, 8192]:
    t_ola = bench(overlap_add, x, h, L)
    t_ols = bench(overlap_save, x, h, L)
    t_np  = bench(np.convolve, x, h, 'full')
    results.append((L, t_ola, t_ols, t_np))
for L, t_ola, t_ols, t_np in results:
    print(f"L={L}: OLA={t_ola:.4f}s, OLS={t_ols:.4f}s, np={t_np:.4f}s")


L=256: OLA=0.0903s, OLS=0.0511s, np=0.0153s
L=512: OLA=0.0602s, OLS=0.0412s, np=0.0354s
L=1024: OLA=0.0946s, OLS=0.0314s, np=0.0234s
L=2048: OLA=0.0571s, OLS=0.0286s, np=0.0247s
L=4096: OLA=0.0322s, OLS=0.0264s, np=0.0217s
L=8192: OLA=0.0276s, OLS=0.0289s, np=0.0241s


In [38]:
"""bonus2"""
def nextpow2(n):
    return 1 << (n-1).bit_length()

def overlap_add(x, h, L):
    M = len(h)
    N = nextpow2(L + M - 1)
    H = np.fft.rfft(h, N)
    y = np.zeros(len(x) + M - 1, dtype=np.float32)
    for k in range(0, len(x), L):
        x_blk = x[k:k+L]
        x_blk_pad = np.pad(x_blk, (0, N - len(x_blk)))
        X = np.fft.rfft(x_blk_pad, N)
        Y_blk = np.fft.irfft(X * H, N)
        valid_len = min(N, len(y) - k)
        y[k:k+valid_len] += Y_blk[:valid_len]
    return y[:len(x) + M - 1]

def overlap_save(x, h, L):
    M = len(h)
    N = nextpow2(L + M - 1)
    H = np.fft.rfft(h, N)

    x_padded = np.concatenate([np.zeros(M-1, dtype=x.dtype), x])

    y_list = []
    for k in range(0, len(x) + M - 1, L):
        x_blk = x_padded[k:k+N]
        if len(x_blk) < N:
            x_blk = np.pad(x_blk, (0, N-len(x_blk)))
        X = np.fft.rfft(x_blk, N)
        Y_blk = np.fft.irfft(X * H, N)
        y_list.append(Y_blk[M-1:M-1+L])


    return np.concatenate(y_list)[:len(x) + M - 1]

x = np.random.randn(2**18).astype(np.float32)

# (A) 超長全1濾波器
h_long = np.ones(2048, dtype=np.float32) / 2048

# (B) Hamming窗低通FIR（101點, 通帶0.2 pi）
h_hamming = firwin(101, cutoff=0.2, window='hamming')

def bench(func, *args):
    t0 = time.perf_counter()
    func(*args)
    t1 = time.perf_counter()
    return t1 - t0
    L = 2048  # Block Size

for h, tag in [(h_long, "all-ones(2048)"), (h_hamming, "hamming(101)")]:
    t_ola = bench(overlap_add, x, h, L)
    t_ols = bench(overlap_save, x, h, L)
    t_np  = bench(np.convolve, x, h, 'full')
    y_ola = overlap_add(x, h, L)
    y_ols = overlap_save(x, h, L)
    y_ref = np.convolve(x, h, 'full')
    print(f"\n[{tag}]")
    print(f"OLA: {t_ola:.4f}s, OLS: {t_ols:.4f}s, np: {t_np:.4f}s")
    print("y_ola", len(y_ola), "y_ols", len(y_ols), "y_ref", len(y_ref))
    print("OLA allclose:", np.allclose(y_ola, y_ref, atol=1e-5))
    print("OLS allclose:", np.allclose(y_ols, y_ref, atol=1e-5))


[all-ones(2048)]
OLA: 0.0228s, OLS: 0.0156s, np: 0.0531s
y_ola 264191 y_ols 264191 y_ref 264191
OLA allclose: True
OLS allclose: True

[hamming(101)]
OLA: 0.0214s, OLS: 0.0183s, np: 0.0112s
y_ola 262244 y_ols 262244 y_ref 262244
OLA allclose: True
OLS allclose: True
